In [1]:
import importlib
import indicators as ind
import numpy as np
import tulipy as ti
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import requests
import json
from keras.initializers import glorot_normal
from keras import regularizers

import keras.optimizers                                                                                                                                                                                                                                                            
from keras.models import Sequential                                                                                                                                                                                                                                                
from keras.layers import Dense,Activation                                                                                                                                                                                                                                          
from keras.layers import LSTM 

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
def get_datadf(symbol):
    raw_data = requests.get("https://query1.finance.yahoo.com/v8/finance/chart/"+ symbol +"?period1=1104517800&period2=1507660200&interval=1d").text
    raw_data = json.loads(raw_data)
    df = raw_data[u'chart'][u'result'][0][u'indicators'][u'quote'][0]
    df['adjclose'] = raw_data[u'chart'][u'result'][0][u'indicators'][u'adjclose'][0][u'adjclose']
    df['timestamp'] = raw_data[u'chart'][u'result'][0][u'timestamp']
    
    data = pd.DataFrame.from_dict(df)
    
    data = data.dropna()
    
    data = data[data.volume > 0]
    
    return data

In [3]:
def process_data(stock_data):
    stock_data.index = pd.to_datetime(stock_data['timestamp'], unit='s').values
    stock_data.drop(['timestamp'], axis=1, inplace=True)

    k = stock_data.adjclose / stock_data.close

    stock_data.high = stock_data.high * k
    stock_data.low = stock_data.low * k
    stock_data.open = stock_data.open * k
    stock_data.close = stock_data.adjclose
    stock_data.volume = pd.to_numeric(stock_data.volume / k)
    stock_data.drop(['adjclose'], axis=1, inplace=True)
    
    return stock_data

In [4]:
def get_indicators(stock_data):
    adx = ind.get_ADX(stock_data.high.values, stock_data.low.values, stock_data.close.values, 13)
    stock_data['adx'] = adx[0]
    stock_data['di_plus'] = adx[1]
    stock_data['di_min'] = adx[2]
    stock_data['aroon_osc'] = ind.get_ArronOSC(stock_data.high.values, stock_data.low.values, 9)
    macd = ind.get_MACD(stock_data.close.values, 13,21,9)
    stock_data['macd'] = macd[0]
    stock_data['macd_signal'] = macd[1]
    stock_data['macd_hist'] = macd[2]
    stock_data['rsi'] = ind.get_RSI(stock_data.close.values, 9)
    stoch_osc = ind.get_StochOsc(stock_data.high.values, stock_data.low.values, stock_data.close.values, 13,4,4)
    stock_data['stoch_k'] = stoch_osc[0]
    stock_data['stoch_d'] = stoch_osc[1]
    bbands = ind.get_BollingerBands(stock_data.close.values, 13,2)
    stock_data['bband'] = bbands[0]
    stock_data['std_up'] = bbands[1]
    stock_data['std_down'] = bbands[2]
    stock_data['obv'] = ind.get_OBV(stock_data.close.values, stock_data.volume.values)
    stock_data['adl'] = ind.get_ADL(stock_data.high.values, stock_data.low.values, stock_data.close.values, stock_data.volume.values)
    stock_data['chaikin'] = ind.get_ChaikinMF(stock_data.close.values, stock_data.low.values, stock_data.high.values, stock_data.volume.values, 13)
    kvi = ind.get_KVI(stock_data.high.values, stock_data.low.values, stock_data.close.values, stock_data.volume.values, 9,13)
    stock_data['kvi'] = kvi[0]
    stock_data['kvi_signal'] = kvi[1]
    
    stock_data.dropna(axis=1, inplace=True)
    
    stock_data.loc[:,'target'] = np.zeros(stock_data.shape[0])
    stock_data.loc[1:,'target'] = np.where(((stock_data.close[1:].values - stock_data.close[:-1].values)/stock_data.close[:-1]) >= 0.02,1,0)

    return stock_data
    

In [5]:
nifty_all = ['3MINDIA.NS','8KMILES.NS','ABB.NS','ACC.NS','AIAENG.NS','APLAPOLLO.NS','AUBANK.NS','AARTIIND.NS','ABAN.NS','ADANIENT.NS','ADANIPORTS.NS','ADANIPOWER.NS','ADANITRANS.NS','ABFRL.NS','ADVENZYMES.NS','AEGISCHEM.NS','AHLUCONT.NS','AJANTPHARM.NS','AKZOINDIA.NS','APLLTD.NS','ALKEM.NS','ALBK.NS','ALLCARGO.NS','AMARAJABAT.NS','AMBUJACEM.NS','ANDHRABANK.NS','APARINDS.NS','APOLLOHOSP.NS','APOLLOTYRE.NS','ARVIND.NS','ASHOKLEY.NS','ASHOKA.NS','ASIANPAINT.NS','ASTRAZEN.NS','ASTRAL.NS','ATUL.NS','AUROPHARMA.NS','AVANTIFEED.NS','DMART.NS','AXISBANK.NS','BASF.NS','BEML.NS','BFUTILITIE.NS','BGRENERGY.NS','BSE.NS','BAJAJCORP.NS','BAJAJELEC.NS','BAJFINANCE.NS','BAJAJFINSV.NS','BAJAJHIND.NS','BAJAJHLDNG.NS','BALKRISIND.NS','BALLARPUR.NS','BALMLAWRIE.NS','BALRAMCHIN.NS','BANKBARODA.NS','BANKINDIA.NS','BATAINDIA.NS','BERGEPAINT.NS','BEL.NS','BHARATFIN.NS','BHARATFORG.NS','BHEL.NS','BPCL.NS','BHARTIARTL.NS','INFRATEL.NS','BHUSANSTL.NS','BIOCON.NS','BIRLACORPN.NS','BLISSGVS.NS','BLUEDART.NS','BLUESTARCO.NS','BBTC.NS','BOMDYEING.NS','BRFL.NS','BOSCHLTD.NS','BRIGADE.NS','BRITANNIA.NS','CARERATING.NS','CCL.NS','CESC.NS','CGPOWER.NS','CRISIL.NS','CADILAHC.NS','CANFINHOME.NS','CANBK.NS','CAPF.NS','CAPLIPOINT.NS','CARBORUNIV.NS','CASTROLIND.NS','CEATLTD.NS','CENTRALBK.NS','CENTURYPLY.NS','CENTURYTEX.NS','CERA.NS','CHAMBLFERT.NS','CHENNPETRO.NS','CHOLAFIN.NS','CIPLA.NS','CUB.NS','COALINDIA.NS','COFFEEDAY.NS','COLPAL.NS','CONCOR.NS','COROMANDEL.NS','CORPBANK.NS','COX&KINGS.NS','CROMPTON.NS','CUMMINSIND.NS','CYIENT.NS','DBREALTY.NS','DBCORP.NS','DCBBANK.NS','DCMSHRIRAM.NS','DLF.NS','DABUR.NS','DALMIABHA.NS','DEEPAKFERT.NS','DELTACORP.NS','DEN.NS','DENABANK.NS','DHFL.NS','DHANUKA.NS','DBL.NS','DISHTV.NS','DIVISLAB.NS','LALPATHLAB.NS','DRREDDY.NS','DREDGECORP.NS','EIDPARRY.NS','EIHOTEL.NS','EDELWEISS.NS','EICHERMOT.NS','EMAMILTD.NS','ENDURANCE.NS','ENGINERSIN.NS','EQUITAS.NS','EROSMEDIA.NS','ESCORTS.NS','ESSELPACK.NS','EVEREADY.NS','EXIDEIND.NS','FEDERALBNK.NS','FINCABLES.NS','FINPIPE.NS','FSL.NS','FORTIS.NS','FCONSUMER.NS','FLFL.NS','GAIL.NS','GEPIL.NS','GET&D.NS','GHCL.NS','GMRINFRA.NS','GVKPIL.NS','GDL.NS','GATI.NS','GILLETTE.NS','GSKCONS.NS','GLAXO.NS','GLENMARK.NS','GODFRYPHLP.NS','GODREJCP.NS','GODREJIND.NS','GODREJPROP.NS','GRANULES.NS','GESHIP.NS','GREAVESCOT.NS','GREENPLY.NS','GRUH.NS','GUJALKALI.NS','GUJFLUORO.NS','GUJGASLTD.NS','GMDCLTD.NS','GNFC.NS','GPPL.NS','GSFC.NS','GSPL.NS','GULFOILLUB.NS','HCLTECH.NS','HDFCBANK.NS','HSIL.NS','HTMEDIA.NS','HATHWAY.NS','HAVELLS.NS','HEIDELBERG.NS','HERITGFOOD.NS','HEROMOTOCO.NS','HEXAWARE.NS','HFCL.NS','HIMATSEIDE.NS','HINDALCO.NS','HCC.NS','HINDCOPPER.NS','HINDPETRO.NS','HINDUNILVR.NS','HINDZINC.NS','HUDCO.NS','HDFC.NS','HDIL.NS','ITC.NS','ICICIBANK.NS','ICICIPRULI.NS','ICRA.NS','IDBI.NS','IDFCBANK.NS','IDFC.NS','IFBIND.NS','IFCI.NS','IL&FSTRANS.NS','IRB.NS','ITDCEM.NS','ITI.NS','IDEA.NS','IGARASHI.NS','INDIACEM.NS','ITDC.NS','IBULHSGFIN.NS','IBREALEST.NS','IBVENTURES.NS','INDIANB.NS','INDHOTEL.NS','IOC.NS','IOB.NS','ICIL.NS','INDOCO.NS','IGL.NS','INDUSINDBK.NS','INFIBEAM.NS','NAUKRI.NS','INFY.NS','INGERRAND.NS','INOXLEISUR.NS','INOXWIND.NS','INTELLECT.NS','INDIGO.NS','IPCALAB.NS','JBCHEPHARM.NS','JKCEMENT.NS','JKIL.NS','JBFIND.NS','JKLAKSHMI.NS','JKTYRE.NS','JMFINANCIL.NS','JSWENERGY.NS','JSWSTEEL.NS','JAGRAN.NS','JAICORPLTD.NS','JISLJALEQS.NS','JPASSOCIAT.NS','JPPOWER.NS','J&KBANK.NS','JETAIRWAYS.NS','JINDALPOLY.NS','JINDALSAW.NS','JSLHISAR.NS','JSL.NS','JINDALSTEL.NS','JCHAC.NS','JUBLFOOD.NS','JUBILANT.NS','JUSTDIAL.NS','JYOTHYLAB.NS','KPRMILL.NS','KPIT.NS','KRBL.NS','KAJARIACER.NS','KALPATPOWR.NS','KANSAINER.NS','KTKBANK.NS','KARURVYSYA.NS','KSCL.NS','KEC.NS','KESORAMIND.NS','KITEX.NS','KOLTEPATIL.NS','KOTAKBANK.NS','KWALITY.NS','L&TFH.NS','LTTS.NS','LICHSGFIN.NS','LAXMIMACH.NS','LAKSHVILAS.NS','LTI.NS','LT.NS','LAURUSLABS.NS','LINDEINDIA.NS','LUPIN.NS','MMTC.NS','MOIL.NS','MRF.NS','MAGMA.NS','MGL.NS','MTNL.NS','M&MFIN.NS','M&M.NS','MAHINDCIE.NS','MHRIL.NS','MANAPPURAM.NS','MRPL.NS','MANPASAND.NS','MARICO.NS','MARKSANS.NS','MARUTI.NS','MFSL.NS','MAXINDIA.NS','MCLEODRUSS.NS','MERCK.NS','MINDTREE.NS','MINDACORP.NS','MINDAIND.NS','MONSANTO.NS','MOTHERSUMI.NS','MOTILALOFS.NS','MPHASIS.NS','MUTHOOTFIN.NS','NATCOPHARM.NS','NBCC.NS','NCC.NS','NHPC.NS','NIITTECH.NS','NLCINDIA.NS','NMDC.NS','NTPC.NS','NH.NS','NATIONALUM.NS','NFL.NS','NBVENTURES.NS','NAVINFLUOR.NS','NAVKARCORP.NS','NAVNETEDUL.NS','NETWORK18.NS','NILKAMAL.NS','OBEROIRLTY.NS','ONGC.NS','OIL.NS','OMAXE.NS','OFSS.NS','ORIENTCEM.NS','ORIENTBANK.NS','PCJEWELLER.NS','PIIND.NS','PNBHOUSING.NS','PNCINFRA.NS','PFS.NS','PTC.NS','PVR.NS','PAGEIND.NS','PARAGMILK.NS','PERSISTENT.NS','PETRONET.NS','PFIZER.NS','PHOENIXLTD.NS','PIDILITIND.NS','PEL.NS','POLARIS.NS','PFC.NS','POWERGRID.NS','PRAJIND.NS','PRESTIGE.NS','PRISMCEM.NS','PGHH.NS','PNB.NS','QUESS.NS','RBLBANK.NS','RADICO.NS','RAIN.NS','RAJESHEXPO.NS','RALLIS.NS','RAMCOSYS.NS','RKFORGE.NS','RCF.NS','RTNPOWER.NS','RAYMOND.NS','REDINGTON.NS','RELAXO.NS','RCOM.NS','RELIANCE.NS','RELINFRA.NS','RNAVAL.NS','RPOWER.NS','RELIGARE.NS','REPCOHOME.NS','RUPA.NS','RECLTD.NS','SHK.NS','SJVN.NS','SKFINDIA.NS','SMLISUZU.NS','SREINFRA.NS','SRF.NS','SADBHAV.NS','SANOFI.NS','SCHAEFFLER.NS','SCHNEIDER.NS','SHARDACROP.NS','SFL.NS','SHILPAMED.NS','SCI.NS','SHOPERSTOP.NS','SHREECEM.NS','RENUKA.NS','SHRIRAMCIT.NS','SRTRANSFIN.NS','SIEMENS.NS','SNOWMAN.NS','SOBHA.NS','SOLARINDS.NS','SOMANYCERA.NS','SONATSOFTW.NS','SOUTHBANK.NS','STARCEMENT.NS','SBIN.NS','SAIL.NS','STRTECH.NS','STAR.NS','SUDARSCHEM.NS','SPARC.NS','SUNPHARMA.NS','SUNTV.NS','SUNDRMFAST.NS','SUNTECK.NS','SUPRAJIT.NS','SUPREMEIND.NS','SUPPETRO.NS','SUVEN.NS','SUZLON.NS','SWANENERGY.NS','SYMPHONY.NS','SYNDIBANK.NS','SYNGENE.NS','TTKPRESTIG.NS','TVTODAY.NS','TV18BRDCST.NS','TVSMOTOR.NS','TVSSRICHAK.NS','TAKE.NS','TNPL.NS','TATACHEM.NS','TATACOFFEE.NS','TATACOMM.NS','TCS.NS','TATAELXSI.NS','TATAGLOBAL.NS','TATAINVEST.NS','TATAMTRDVR.NS','TATAMOTORS.NS','TATAPOWER.NS','TATASPONGE.NS','TATASTEEL.NS','TECHM.NS','TECHNO.NS','TEXRAIL.NS','RAMCOCEM.NS','THERMAX.NS','THOMASCOOK.NS','THYROCARE.NS','TIMETECHNO.NS','TIMKEN.NS','TITAN.NS','TORNTPHARM.NS','TORNTPOWER.NS','TRENT.NS','TRIDENT.NS','UCOBANK.NS','UFLEX.NS','UPL.NS','UJJIVAN.NS','ULTRACEMCO.NS','UNICHEMLAB.NS','UNIONBANK.NS','UNITECH.NS','UNITEDBNK.NS','UBL.NS','VGUARD.NS','VIPIND.NS','VRLLOG.NS','WABAG.NS','VAKRANGEE.NS','VTL.NS','VBL.NS','VEDL.NS','VESUVIUS.NS','VIDEOIND.NS','VIJAYABANK.NS','VINATIORGA.NS','VOLTAS.NS','WABCOINDIA.NS','WELCORP.NS','WELSPUNIND.NS','WHIRLPOOL.NS','WIPRO.NS','WOCKPHARMA.NS','YESBANK.NS','ZEEL.NS','ZEELEARN.NS','ZENSARTECH.NS','ECLERX.NS','ADANIPORTS.NS','ZEEL.NS'] 

In [8]:
stocks = pd.DataFrame()

In [9]:
for stock_sym in nifty_all[:100]:
    stock_data = get_datadf(stock_sym)
    stock_data = process_data(stock_data)
    stock_data = get_indicators(stock_data)
    
    print(stock_sym)
    stocks = stocks.append(stock_data)

/home/rushim245/DeepLearning/indicators.py:66: RuntimeWarning: invalid value encountered in true_divide
  money_flow_mul = ((close - low) - (high - close)) / (high -low)


3MINDIA.NS
8KMILES.NS
ABB.NS
ACC.NS
AIAENG.NS
APLAPOLLO.NS
AUBANK.NS


/home/rushim245/DeepLearning/indicators.py:66: RuntimeWarning: divide by zero encountered in true_divide
  money_flow_mul = ((close - low) - (high - close)) / (high -low)


AARTIIND.NS
ABAN.NS
ADANIENT.NS
ADANIPORTS.NS
ADANIPOWER.NS
ADANITRANS.NS
ABFRL.NS
ADVENZYMES.NS
AEGISCHEM.NS
AHLUCONT.NS
AJANTPHARM.NS
AKZOINDIA.NS
APLLTD.NS
ALKEM.NS
ALBK.NS
ALLCARGO.NS
AMARAJABAT.NS
AMBUJACEM.NS
ANDHRABANK.NS
APARINDS.NS
APOLLOHOSP.NS
APOLLOTYRE.NS
ARVIND.NS
ASHOKLEY.NS
ASHOKA.NS
ASIANPAINT.NS
ASTRAZEN.NS
ASTRAL.NS
ATUL.NS
AUROPHARMA.NS
AVANTIFEED.NS
DMART.NS
AXISBANK.NS
BASF.NS
BEML.NS
BFUTILITIE.NS
BGRENERGY.NS
BSE.NS
BAJAJCORP.NS
BAJAJELEC.NS
BAJFINANCE.NS
BAJAJFINSV.NS
BAJAJHIND.NS
BAJAJHLDNG.NS
BALKRISIND.NS
BALLARPUR.NS
BALMLAWRIE.NS
BALRAMCHIN.NS
BANKBARODA.NS
BANKINDIA.NS
BATAINDIA.NS
BERGEPAINT.NS
BEL.NS
BHARATFIN.NS
BHARATFORG.NS
BHEL.NS
BPCL.NS
BHARTIARTL.NS
INFRATEL.NS
BHUSANSTL.NS
BIOCON.NS
BIRLACORPN.NS
BLISSGVS.NS
BLUEDART.NS
BLUESTARCO.NS
BBTC.NS
BOMDYEING.NS
BRFL.NS
BOSCHLTD.NS
BRIGADE.NS
BRITANNIA.NS
CARERATING.NS
CCL.NS
CESC.NS
CGPOWER.NS
CRISIL.NS
CADILAHC.NS
CANFINHOME.NS
CANBK.NS
CAPF.NS
CAPLIPOINT.NS
CARBORUNIV.NS
CASTROLIND.NS
CEATLTD.NS
CENT

In [11]:
stocks.head()

,adx,aroon_osc,bband,close,di_min,di_plus,high,kvi,low,macd,macd_hist,macd_signal,open,rsi,std_down,std_up,stoch_d,stoch_k,target,volume
2005-01-03 03:45:00,0.0,0.0,0.0,581.099976,0.0,0.0,585.000000,NaN,580.000000,0.0,0.0,0.0,584.000000,0.0,0.0,0.0,0.0,0.0,0.0,1387.0
2005-01-04 03:45:00,0.0,0.0,0.0,570.000000,0.0,0.0,579.950012,NaN,565.549988,0.0,0.0,0.0,566.599976,0.0,0.0,0.0,0.0,0.0,0.0,376.0
2005-01-05 03:45:00,0.0,0.0,0.0,550.000000,0.0,0.0,560.000000,NaN,550.000000,0.0,0.0,0.0,560.000000,0.0,0.0,0.0,0.0,0.0,0.0,1013.0
2005-01-06 03:45:00,0.0,0.0,0.0,550.049988,0.0,0.0,560.000000,NaN,540.849976,0.0,0.0,0.0,560.000000,0.0,0.0,0.0,0.0,0.0,0.0,814.0
2005-01-07 03:45:00,0.0,0.0,0.0,561.000000,0.0,0.0,590.000000,NaN,560.000000,0.0,0.0,0.0,590.000000,0.0,0.0,0.0,0.0,0.0,0.0,313.0


In [12]:
cols = [col for col in stocks.columns if col not in ['kvi','target']]
X = stocks.loc[:,cols].values
Y = stocks['target'].values

In [13]:
np.sum(stocks.isnull(), axis=0)

adx                0
aroon_osc          0
bband              0
close              0
di_min             0
di_plus            0
high               0
kvi            13589
low                0
macd               0
macd_hist          0
macd_signal        0
open               0
rsi                0
std_down           0
std_up             0
stoch_d            0
stoch_k            0
target             0
volume             0
dtype: int64

In [14]:
X.shape

(261395, 18)

In [15]:
nb_epoch = 10                                                                                                                                                                                                                                                                 
batch_size = 100                                                                                                                                                                                                                                                                  
learning_rate = 0.01                                                                                                                                                                                                                                                               
nb_units = 50                                                                                                                                                                                                                                                                 
timeStep = 15

In [16]:
from sklearn.preprocessing import StandardScaler
stdscl = StandardScaler()

In [17]:
X = stdscl.fit_transform(X)
Y = stocks.target.values

In [18]:
new_X = np.zeros([X.shape[0]-timeStep, timeStep, X.shape[1]])
new_Y = np.zeros([X.shape[0] - timeStep])

In [19]:
for i in range(0, X.shape[0]-timeStep):
    new_X[i] = X[i:i+timeStep,:]
    new_Y[i] = Y[i+timeStep-1]

In [20]:
new_X.shape

(261380, 15, 18)

In [21]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(new_X,new_Y, test_size=0.2)

In [22]:
trainX.shape

(209104, 15, 18)

In [23]:
model = Sequential()                                                                                                                                                                                                                                                               
model.add(LSTM(input_dim = X.shape[1],output_dim = nb_units,return_sequences = False,  kernel_initializer=glorot_normal(seed=22), kernel_regularizer=regularizers.l2(0.001))) # One LSTM layer with 50 units                                                                                                                                                               
model.add(Activation("relu"))     

model.add(Dense(50, activation='relu',  kernel_initializer=glorot_normal(seed=22), kernel_regularizer=regularizers.l2(0.001)))
model.add(Dense(1, activation='sigmoid')) #A dense layer which is the final layer                                                                                                                                                                                                                         

/home/rushim245/anaconda2/envs/DeepLearning/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
/home/rushim245/anaconda2/envs/DeepLearning/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=False, kernel_initializer=<keras.ini..., kernel_regularizer=<keras.reg..., input_shape=(None, 18), units=50)`
  


In [24]:
KerasOptimizer = keras.optimizers.Adam(lr=learning_rate, epsilon=1e-08, decay=0.0)                                                                                                                                                                                     
model.compile(loss="binary_crossentropy", optimizer=KerasOptimizer, metrics=['accuracy'])                                                                                                                                                                                                                                
model.fit(trainX,trainY,epochs = nb_epoch,batch_size = batch_size, verbose=2)                                                                                                                                                                                                                         


Epoch 1/10
78s - loss: 0.2093 - acc: 0.9247
Epoch 2/10
77s - loss: 0.1687 - acc: 0.9374
Epoch 3/10
77s - loss: 0.1635 - acc: 0.9390
Epoch 4/10
77s - loss: 0.1760 - acc: 0.9388
Epoch 5/10
77s - loss: 0.1641 - acc: 0.9395
Epoch 6/10
77s - loss: 0.1660 - acc: 0.9403
Epoch 7/10
77s - loss: 0.1602 - acc: 0.9403
Epoch 8/10
77s - loss: 0.1553 - acc: 0.9412
Epoch 9/10
78s - loss: 0.1547 - acc: 0.9424
Epoch 10/10
77s - loss: 0.1497 - acc: 0.9426


In [25]:
prediction = model.predict(trainX)
pred = np.where(prediction > 0.5, 1,0)
pd.crosstab(pred.reshape(-1), trainY)

col_0,0.0,1.0
row_0,,
0,168326,7159
1,4697,28922


In [26]:
from sklearn.metrics import precision_recall_fscore_support

In [27]:
precision_recall_fscore_support(trainY,pred.reshape(-1) )

(array([ 0.95920449,  0.86028734]),
 array([ 0.97285332,  0.80158532]),
 array([ 0.96598069,  0.82989957]),
 array([173023,  36081]))

In [29]:
prediction = model.predict(testX)
pred = np.where(prediction > 0.5, 1,0)
pd.crosstab(pred.reshape(-1), testY)

col_0,0.0,1.0
row_0,,
0,42064,1798
1,1233,7181


In [30]:
precision_recall_fscore_support(testY,pred.reshape(-1) )

(array([ 0.9590078 ,  0.85345852]),
 array([ 0.97152228,  0.79975498]),
 array([ 0.96522447,  0.82573449]),
 array([43297,  8979]))